In [28]:
!pip install PyExecJS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for PyExecJS: filename=PyExecJS-1.5.1-py3-none-any.whl size=14597 sha256=eff8035010b53def1042b94865b2fc7c157fc7aacf03452ee66315a070d3a967
  Stored in directory: /root/.cache/pip/wheels/9a/ee/03/da5c0b4a8c13362beeb844eb913bbe58a89bde1de2b9157007
Successfully built PyExecJS


In [45]:
import execjs

with open('./base64.js', 'r') as f:
  content = f.read()
Base64 = execjs.compile(content)

with open('./des.js', 'r') as f:
  content = f.read()
DES = execjs.compile(content)

with open('./md5.js', 'r') as f:
  content = f.read()
MD5 = execjs.compile(content)

In [50]:
import requests
from bs4 import BeautifulSoup
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.42'
})

url = 'https://elearning.nkust.edu.tw/mooc/login.php'

web = session.get(url, verify=False)
soup = BeautifulSoup(web.text, "html5lib")

In [51]:
MY_USERNAME = '<your-student-id>'
MY_PASSWD = '<your-password>'

node = soup.find(id="loginForm")

pwdmask = "********************************"
login_key = node.find('input', {'name':'login_key'}).get('value')
md5key  = MD5.call('MD5', MY_PASSWD)
cypkey  = md5key[:4] + login_key[:4]
passwd = Base64.call('stringToBase64', MY_PASSWD)
encrypt_pwd = Base64.call('stringToBase64', DES.call('des', cypkey, MY_PASSWD, 1))
password = pwdmask[:len(MY_PASSWD)]

In [52]:
url = 'https://elearning.nkust.edu.tw/login.php'

res = session.post('http://httpbin.org/post', data={
  'login_key': login_key,
  'encrypt_pwd': encrypt_pwd,
  'passwd': passwd,
  'username': MY_USERNAME,
  'password': password,
})

res.status_code

200

In [53]:
res = session.get('https://elearning.nkust.edu.tw/logout.php')
session.close

res.status_code

200